In [36]:
#to do list:
# write a code for a start button the begin the test
# format the questions in a pretty way
# finish creating all the questions
# link to a google form
# add a way of allowing users to view their performance in comparison to the class dataset results
# make interface more pretty
#fix timing and score - sometimes timing is negative, sometimes score is over 100%

In [37]:
# importing the python libraries
from IPython.display import display, Image, clear_output, HTML, Audio
from jupyter_ui_poll import ui_events
import ipywidgets as widgets
import time
import random
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json 
import warnings

In [38]:
def register_text_input_event(text_input):
    event_info['type'] = "text_entry"
    event_info['description'] = text_input.value
    event_info['time'] = time.time()
    return

def text_input(prompt=None):
    text_input = widgets.Text(description=prompt, style= {'description_width': 'initial'})
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    text_input.on_submit(register_text_input_event)
    display(text_input)
    event = wait_for_event()
    text_input.disabled = True
    return event['description']

In [39]:
# Function to send data to Google Form

def send_to_google_form(data_dict, form_url):

    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok


'''def send_to_google_form(data_dict, form_url):
    form_id = form_url.split('/d/')[1].split('/viewform')[0]
    view_form_url = f'https://docs.google.com/forms/d/e/1FAIpQLScLFJvQWuQ4voGLWNRkINu0vr1j35uWCZ3-zriKamc09tYpfg/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/1FAIpQLScLFJvQWuQ4voGLWNRkINu0vr1j35uWCZ3-zriKamc09tYpfg/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}

    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]

    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok'''

# Function for waiting for an event
def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # reset event info
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:

            # process UI events
            ui_poll(n_proc)

            # end the loop if the time spent looping is longer than the timeout period. -1 can be set for no timeout
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False

            # when the event_info description changes, stop looping
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False

            time.sleep(interval)
    
    return event_info

# Function to register button event
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()

# Function to display widgets centered
def display_centered(widget):
    box_layout = widgets.Layout(display='flex', justify_content='center', align_items='center')
    box = widgets.Box([widget], layout=box_layout)
    display(box)

# Callback function for submit button
def submit_button_callback():
    global gender, age
    age = age_dropdown.value
    gender = gender_radiobuttons.value
    # Call send_to_google_form here with correct data and URL
    clear_output(wait=False)

#used to display a message for the user with a countdown feature in it
def timer(message):
    for i in range(5, 0, -1):
        clear_output(wait=True)
        formatted_message = message.format(i)
        style = "color: orchid; font-size: 100px; display: flex; justify-content: center; align-items: center;"
        display(HTML(f"<div style='{style}'><span>{formatted_message}</span></div>"))
        time.sleep(1)
    clear_output(wait=True)
timer_message = "Beginning the test in... {}"

# Function to display the submit button
def submit_button(age_dropdown, gender_radiobuttons):
    confirm_btn = widgets.Button(description="Confirm and Begin")
    display_centered(confirm_btn)
    confirm_btn.on_click(register_btn_event)
    wait_for_event()
    submit_button_callback()
    clear_output(wait=False)


# Main function to get personal information
def personal_info():
    personal_info_1 = 'Welcome to the Arithmetic test. This test will take approximately 3 minutes and the questions will progressively increase in difficulty.'
    personal_info_2 = 'Each part of the question will flash on your screen for 1.5 seconds after which you will be prompted to respond.'
    personal_info_3 = 'Your final score will be calculated based on the time taken to answer each question and the number of correct answers, so try to be quick and precise!'
    style_personal_info = "color: cornflowerblue; font-size: 20px; display: flex; justify-content: center; align-items: center;"
    display(HTML(f"<div style='{style_personal_info}'><span>{personal_info_1}</span></div>"))
    display(HTML(f"<div style='{style_personal_info}'><span>{personal_info_2}</span></div>"))
    display(HTML(f"<div style='{style_personal_info}'><span>{personal_info_3}</span></div>"))
    time.sleep(3)
    personal_info_4 = 'First, please can you provide some background information for an anonymised study. Filling out this form will be considered consent for data collection. Thank you!'
    style_personal_info_2 = "color: plum; font-size: 20px; display: flex; justify-content: center; align-items: center;"
    display(HTML(f"<div style='{style_personal_info_2}'><span>{personal_info_4}</span></div>"))
    time.sleep(2)

    personal_info_5 = 'How old are you?'
    display(HTML(f"<div style='{style_personal_info_2}'><span>{personal_info_5}</span></div>"))
    display_centered(age_dropdown)

    personal_info_5 = 'What is your gender?'
    display(HTML(f"<div style='{style_personal_info_2}'><span>{personal_info_5}</span></div>"))
    display_centered(gender_radiobuttons)

    submit_button(age_dropdown, gender_radiobuttons)
    clear_output(wait=False)

    personal_info_6 = 'Thank you! The test will begin shortly.'
    display(HTML(f"<div style='{style_personal_info}'><span>{personal_info_6}</span></div>"))
    time.sleep(2)
    clear_output(wait=False)

    timer(timer_message)
    global test_start_time
    test_start_time = time.time() 

#fix this!!!!!

In [40]:
def upload_data():
    
    results_data = {
    
        'answer': users_answers
    }
    
    myresults = pd.DataFrame(results_data)
    results_json = myresults.to_json()
    
    
    data_dict = {
        'gender': gender,
        'age': age,
        'results': results_json, 
        'test_time': total_test_time,
        'score_as_percent': score_percentage,
        'score_with_penalty': score_time
        }
    
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSeec7lgksjIePCmUxYP3KOo3Ww2NdeytOsWx-K2UiqQDbEfaw/viewform?usp=sf_link' 

    send_to_google_form(data_dict, form_url)

In [41]:

def clear_screen():
    """
    Clears the output screen based on the operating system.
    """
    os.system('cls' if os.name == 'nt' else 'clear')

# Function to handle the submission of an answer and ensure the code continues running if a non-numerical value is input
def on_submit(their_answer):
    global correct, incorrect, times, current_index, start_time, users_answers
    time_to_answer = time.time() - start_time  # Time taken for this question
    times.append(time_to_answer)  # Append time for this question
    
    if their_answer == None:
        error_handling_style = "color: tomato; font-size: 20px; display: flex; justify-content: center; align-items: center;"
        error_handling= 'Invalid input entered. Please only type numbers :)! Try again.'
        display(HTML(f"<div style='{error_handling_style}'><span>{error_handling}</span></div>"))

        # Here we don't increment the current_index as we're retrying the current question.
        time.sleep(2)
        clear_output(wait=True)
        display_question(current_index)  # Display next question
    else:
        time_to_answer = time.time() - start_time
        times.append(time_to_answer)
    
        style_personal_info = "color: skyblue; font-size: 20px; display: flex; justify-content: center; align-items: center;"

    if their_answer == answers[current_index]:
        correct_answer_response = f"{answers[current_index]} is correct! :)"
        display(HTML(f"<div style='{style_personal_info}'><span>{correct_answer_response}</span></div>"))
        correct += 1
        users_answers.append(1)
    else:
        incorrect_answer_response = f"Incorrect :( The correct answer was: {answers[current_index]}"
        display(HTML(f"<div style='{style_personal_info}'><span>{incorrect_answer_response}</span></div>"))
        incorrect += 1
        users_answers.append(0)
        
    if current_index < len(questions) - 1:
        current_index += 1
        time.sleep(2)
        clear_output(wait=True)
        display_question(current_index)  # Display next question
    else:
        calculate_score()

# Function to display the question using widgets
def display_question(index):
    """
    Displays each component of the question.
    
    Parameters:
    index: int, the index of the current question
    """
    global start_time
    question = questions[index]
    start_time = time.time()
    for part in question.split():
        style = "color: palevioletred; font-size: 100px; display: flex; justify-content: center; align-items: center;"
        display(HTML(f"<div style='{style}'><span>{part}</span></div>"))
        time.sleep(1.5)
        clear_output(wait=True)

    answer_input = text_input("Answer: ")


    if not answer_input.lstrip("-").isdigit():
        on_submit(None)
    else :
        ans_input_int = int(answer_input)
        on_submit(ans_input_int)

# Function to calculate and display the final score
def calculate_score():
    global total_test_time
    global score_percentage
    global score_time
    """
    Calculates the final score based on the number of correct and incorrect answers and the time taken.
    """
    total_test_time = time.time() - test_start_time  # Total time for the test
    print(f"Test completed in {total_test_time:.2f} seconds.")
    score_percentage = (correct / (correct + incorrect)) * 100
    print(f"You got: {correct} questions correct out of {len(questions)} ({score_percentage:.2f}%)")

    hypothesis_time = hypothesis_time_per_question * len(questions)
    time_difference = total_test_time - hypothesis_time

    # Apply penalty only if the total time taken is more than the hypothetical time
    penalty = max(time_difference, 0) * 0.01  # 0.01 as the penalty rate

    # Calculate final score
    score_time = ((correct - penalty) / (correct + incorrect)) * 100
    print(f"Accounting for the time taken, your final score is {score_time:.2f}%! :)")

# Main test function
def run_math():
    personal_info()
    
    """
    Runs the math test by displaying each question and collecting answers.
    """
    display_question(0)  # Display first question
    upload_data()




# Questions and answers


event_info = {
    'type': '',
    'description': '',
    'time': -1}
age_dropdown = widgets.Dropdown(
        options=[('Select Age', None)] + [(str(age), age) for age in range(18, 30)],
        value=None
    )
gender_radiobuttons = widgets.RadioButtons(
        options=['Male', 'Female', 'Other'],
        description='',
        disabled=False
    )



In [42]:
# Global variables
correct = 0
incorrect = 0
times = []
current_index = 0
hypothesis_time_per_question = 4
start_time = 0  # Start time for each question
test_start_time = 0  # Start time for the entire test
users_answers=[]
questions = ['9 + 7', '13 - 20', '34 + 21', '56 - 28', '27 ÷ 9',
    '5 × 8', '56 ÷ 7', '8 × 11', '144 ÷ 12', '184 + 49',
    '135 ÷ 3', '13 × 7', '17 × 12', '119 + 37', '1091 - 195',
    '112 ÷ 16', '703 - 339']

answers = [16, -7, 55, 28, 3,
    40, 8, 88, 12, 233,
    45, 91, 204, 156, 896,
    7, 364]






run_math()

Text(value='', description='Answer: ', style=DescriptionStyle(description_width='initial'))

KeyboardInterrupt: 